In [0]:
import json
import requests
import time

# --- Configuration ---
DASHBOARD_NAME = "Paas_card_kgsp_V2" # Update name of your choice here 
WAREHOUSE_ID = "ead91b1206ad0b7e" # for this id only the part before ? in the warehouse info-  https://dbc-2c2381f1-79ce.cloud.databricks.com/sql/warehouses/6ffec9dd3f6ddc56?o=560907530345231
UPDATE_IF_EXISTS = True  # Set to False to create new dashboard with timestamp

# --- Setup ---
ctx = dbutils.notebook.entry_point.getDbutils().notebook().getContext()
host = ctx.apiUrl().get()
token = ctx.apiToken().get()
headers = {"Authorization": f"Bearer {token}", "Content-Type": "application/json"}

file_path = "/Volumes/ds_training_1/thoughtspot_inventory_ak/lvdash_files_ak_out/PaaS Tracking Card.liveboard.lvdash.json" # replace with the volume path w copied now.

print(f"Authenticated to {host}")

# --- Check for existing dashboard ---
print(f"Checking for existing dashboards...")
list_url = f"{host}/api/2.0/lakeview/dashboards"

existing_dashboard = None
list_resp = requests.get(list_url, headers=headers)

if list_resp.status_code == 200:
    dashboards = list_resp.json().get("dashboards", [])
    
    for dash in dashboards:
        if dash.get("display_name") == DASHBOARD_NAME:
            existing_dashboard = dash
            break
    
    if existing_dashboard:
        dashboard_id = existing_dashboard.get("dashboard_id")
        etag = existing_dashboard.get("etag")
        print(f"Dashboard '{DASHBOARD_NAME}' already exists with ID: {dashboard_id}")
        
        if not UPDATE_IF_EXISTS:
            DASHBOARD_NAME = f"{DASHBOARD_NAME}_{int(time.time())}"
            print(f"Creating new dashboard with name: {DASHBOARD_NAME}")
            existing_dashboard = None
        else:
            print(f"Updating existing dashboard...")

# --- Create dashboard if needed ---
if not existing_dashboard or not UPDATE_IF_EXISTS:
    print(f"Creating new dashboard '{DASHBOARD_NAME}'...")
    create_url = f"{host}/api/2.0/lakeview/dashboards"
    create_payload = {
        "display_name": DASHBOARD_NAME,
        "warehouse_id": WAREHOUSE_ID
    }
    
    create_resp = requests.post(create_url, headers=headers, json=create_payload)
    
    if create_resp.status_code not in [200, 201]:
        raise Exception(f"Dashboard creation failed: {create_resp.status_code} - {create_resp.text}")
    
    dashboard_data = create_resp.json()
    dashboard_id = dashboard_data.get("dashboard_id")
    etag = dashboard_data.get("etag")
    print(f"Dashboard created with ID: {dashboard_id}")

# --- Read dashboard content ---
print(f"Reading dashboard definition from file...")
dashboard_json_str = dbutils.fs.head(file_path, 10000000)
dashboard_json_content = json.loads(dashboard_json_str)
print(f"Dashboard definition loaded successfully")

# --- Update dashboard ---
print(f"Updating dashboard {dashboard_id}...")
update_url = f"{host}/api/2.0/lakeview/dashboards/{dashboard_id}"

update_payload = {
    "display_name": DASHBOARD_NAME,
    "warehouse_id": WAREHOUSE_ID
}

# Serialize dashboard content to JSON string
if "serialized_dashboard" in dashboard_json_content:
    serialized_content = dashboard_json_content["serialized_dashboard"]
    if isinstance(serialized_content, str):
        update_payload["serialized_dashboard"] = serialized_content
    else:
        update_payload["serialized_dashboard"] = json.dumps(serialized_content)
else:
    update_payload["serialized_dashboard"] = json.dumps(dashboard_json_content)

# Add etag for version control
headers_with_etag = headers.copy()
if etag:
    headers_with_etag["If-Match"] = etag

update_resp = requests.patch(update_url, headers=headers_with_etag, json=update_payload)

if update_resp.status_code != 200:
    raise Exception(f"Dashboard update failed: {update_resp.status_code} - {update_resp.text}")

print(f"Dashboard updated successfully")
print(f"Dashboard ID: {dashboard_id}")
print(f"Dashboard URL: {host}/sql/dashboardsv3/{dashboard_id}")

# CODE YET TO BE TESTED FOR PUBISH - UNABLE TO PUBLISH AS OF NOW FROM TRIAL ACC?

# # --- Publish dashboard ---
# print(f"Publishing dashboard...")
# publish_url = f"{host}/api/2.0/lakeview/dashboards/{dashboard_id}/published"
# publish_payload = {
#     "warehouse_id": WAREHOUSE_ID,
#     "embed_credentials": False
# }

# publish_resp = requests.put(publish_url, headers=headers, json=publish_payload)

# if publish_resp.status_code == 200:
#     print(f"Dashboard published successfully")
# else:
#     print(f"Dashboard remains in draft mode: {publish_resp.status_code}")

current_time = time.strftime("%Y-%m-%d %H:%M:%S")
print("Process completed")
print(current_time)